# This Lab will compare proximal method and line search method for least squares problem with and without regularization

In [57]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


# Loading raw data
import pandas as pd
data = pd.read_csv('Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')


Question 3.1

We suppose $Aw=b $

Then $Aw_t=b_t=y(t)=\tilde{x}(t)^Tw_1+w_0-y(t)\tilde{x}(t)^Tw_2$

So $y(t)(1+w_2^T\tilde{x}(t))=w_1^T\tilde{x}(t)+w_0$

i.e $y(t)=\frac {w_1^T\tilde{x}(t)+w_0} {1+w_2^T\tilde{x}(t)}$

In [58]:
#Question 3.2
w,res,r,s=np.linalg.lstsq(A,b)
w0,w1,w2=w[0],w[1:np.shape(w)[0]//2+1],w[np.shape(w)[0]//2+1:]
#Question 3.3
error=np.linalg.norm(A_test@w-b_test)/np.shape(b_test)[0]
print(error)
# The result is greater than 1, in average the difference between two coefficients of Aw and b is greater than 1

C:\Users\fanto\AppData\Local\Temp\ipykernel_24920\1786724896.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w,res,r,s=np.linalg.lstsq(A,b)


1.4707663588810986


In [59]:
#Question 3.4
from sklearn.linear_model import Ridge
ldb=100
clf=Ridge(alpha=ldb)
clf.fit(A,b)
error=np.linalg.norm(clf.predict(A_test)-b_test)/np.shape(b_test)[0]
print(error)
clf.score(A_test,b_test)
# We get a better result but the negative score function indicates that a constant approximation is still better than the Ridge method

0.42012531159394634


-2.416913745569063

Question 3.5

$\nabla f_1(x)=A^T(Ax-b) + \lambda x$

$\nabla^2 f_1 = A^TA + \lambda Id$ is positive definite because $<(A^TA+Id)x,x>=<Ax,Ax>+<x,x> \gt 0$ if $x \neq 0$

For the next question we use the following inequality :  $ \lVert \nabla^2 f_1(x) - \nabla^2 f_1(y) \rVert \leq \lVert A^TA+Id\rVert \lVert x-y\rVert$  so the gradient descent converges if $\gamma \leq \frac 2 { \lVert A^TA+Id\rVert} $

In [60]:
#Question 3.6

L=np.linalg.norm(np.transpose(A)@A+ldb*np.eye(np.shape(A)[1]))
gamma=1/L

def grad_f1(x):
    return np.transpose(A)@(A@x-b)+ldb*x

def descent_grad(gamma,epsilon,w):
    n_iter=0
    while np.linalg.norm(grad_f1(w))>1:
        w=w-gamma*grad_f1(w)
        n_iter+=1
        print(np.linalg.norm(grad_f1(w)))
    return w,n_iter

w_grad,n=descent_grad(gamma,1,w)
print(n)
#gradient descent : 181701 iteration, 6 min

Question 4.1
$$f_2 = \lVert Ax - b \rVert $$
 $$g_2 = \lambda \lVert x\rVert _1$$ 
 $$prox_{g_2}(y)= (sign(y_1)(| y_1  | - \lambda )_{+}, ... ,sign(y_n)(| y_n  | - \lambda )_{+})$$
 $$\nabla f_2(x)=A^T(Ax-b)$$

In [63]:
#Question 4.2
import math
sign = lambda x : math.copysign(1,x)

ldb = 200

def prox_g(y,alpha):
    res=np.zeros(len(y))
    for i in range(len(y)):
        res[i]=sign(y[i])*np.max(np.array([(np.abs(y[i])-alpha*ldb),0]))
    return res


def grad_f2(x):
    return np.transpose(A)@(A@x-b)

def stop_cond(x1,x2,L):
    print(L*np.linalg.norm(x1-x2))
    return L*np.linalg.norm(x1-x2)<=2

def prox_method(gamma,epsilon,w):
    w1=w
    w2=2*w
    n_iter=0
    while not stop_cond(w1,w2,1/gamma):
        w2=w1
        w1=prox_g(w1-gamma*grad_f2(w1),gamma)
        n_iter+=1
    return w,n_iter

def prox_line_method(gamma,epsilon,w):
    a=1/2
    w1=w
    w2=2*w
    n_iter=0
    while not stop_cond(w1,w2,1/gamma):
        w2=w1
        w1=prox_g(w1-gamma*grad_f2(w1),gamma*a**n_iter)
        n_iter+=1
    return w,n_iter


13138318.568454586
8444.201241415993
4211.9696815284005
2103.4129293871406
1050.7714015851236
526.2437078125688
266.4287225851711
139.91361893681395
80.98387710721133
55.558759885485195
44.89058805096769
39.69593820369758
36.43732979753845
34.00072314976567
32.03949876719041
30.419508281304594
29.065265715411734
27.92305047695439
26.94897696443847
26.108551268373407
25.37450006185861
24.725505326123443
24.145018646969138
23.620222644649857
23.14117383829197
22.700123523716833
22.290993560968886
21.908978068661114
21.550243355319786
21.211702623523532
20.89084676357826
20.585616893220937
20.29430787731265
20.01549484231909
19.74797681258114
19.49073315004602
19.24288963042246
19.003691815561325
18.77248399573441
18.548692412696575
18.331811798123642
18.12139450053847
17.917041645096933
17.718395902000253
17.52513553608462
17.336969483995283
17.15363325662391
16.974885513262983
16.800505180599135
16.63028901847206
16.464049553459095
16.30161331440074
16.14281932087672
15.9875177809669
15

In [ ]:
w_grad,n=prox_method(gamma,1,w)
print(n)

# Proximal method : 35261 iteration , 9 min

In [ ]:
w_grad,n=prox_line_method(gamma,1,w)
print(n)

# Line search + Proximal method : 801 iteration, 8 seconds

Question 6.3

Proximal method : 35261 iteration , 9 min

Line search + Proximal method : 801 iteration, 8 seconds

Gradient descent : 181701 iteration, 6 min